## 📈 Startup Funding Prediction

Given *data about startups in India*, let's try to predict the **funding** provided to a given startup.

We will use TensorFlow/Keras neural network within a scikit-learn pipeline to make our predictions.

Data source: https://www.kaggle.com/datasets/sudalairajkumar/indian-startup-funding

### Getting Started

In [2]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

import tensorflow as tf

from sklearn.metrics import r2_score

In [3]:
data = pd.read_csv('archive/startup_funding.csv')
data

,Sr No,Date dd/mm/yyyy,Startup Name,Industry Vertical,SubVertical,City Location,Investors Name,InvestmentnType,Amount in USD,Remarks
0,1,09/01/2020,BYJU’S,E-Tech,E-learning,Bengaluru,Tiger Global Management,Private Equity Round,"20,00,00,000",NaN
1,2,13/01/2020,Shuttl,Transportation,App based shuttle service,Gurgaon,Susquehanna Growth Equity,Series C,"80,48,394",NaN
2,3,09/01/2020,Mamaearth,E-commerce,Retailer of baby and toddler products,Bengaluru,Sequoia Capital India,Series B,"1,83,58,860",NaN
3,4,02/01/2020,https://www.wealthbucket.in/,FinTech,Online Investment,New Delhi,Vinod Khatumal,Pre-series A,"30,00,000",NaN
4,5,02/01/2020,Fashor,Fashion and Apparel,Embroiled Clothes For Women,Mumbai,Sprout Venture Partners,Seed Round,"18,00,000",NaN
...,...,...,...,...,...,...,...,...,...,...
3039,3040,29/01/2015,Printvenue,NaN,NaN,NaN,Asia Pacific Internet Group,Private Equity,"45,00,000",NaN
3040,3041,29/01/2015,Graphene,NaN,NaN,NaN,KARSEMVEN Fund,Private Equity,"8,25,000",Govt backed VC Fund
3041,3042,30/01/2015,Mad Street Den,NaN,NaN,NaN,"Exfinity Fund, GrowX Ventures.",Private Equity,"15,00,000",NaN
3042,3043,30/01/2015,Simplotel,NaN,NaN,NaN,MakeMyTrip,Private Equity,NaN,"Strategic Funding, Minority stake"


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3044 entries, 0 to 3043
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Sr No              3044 non-null   int64 
 1   Date dd/mm/yyyy    3044 non-null   object
 2   Startup Name       3044 non-null   object
 3   Industry Vertical  2873 non-null   object
 4   SubVertical        2108 non-null   object
 5   City  Location     2864 non-null   object
 6   Investors Name     3020 non-null   object
 7   InvestmentnType    3040 non-null   object
 8   Amount in USD      2084 non-null   object
 9   Remarks            419 non-null    object
dtypes: int64(1), object(9)
memory usage: 237.9+ KB


### Preprocessing

In [5]:
df = data.copy()

In [6]:
df.isna().mean()

Sr No                0.000000
Date dd/mm/yyyy      0.000000
Startup Name         0.000000
Industry Vertical    0.056176
SubVertical          0.307490
City  Location       0.059133
Investors Name       0.007884
InvestmentnType      0.001314
Amount in USD        0.315375
Remarks              0.862352
dtype: float64

In [7]:
df

,Sr No,Date dd/mm/yyyy,Startup Name,Industry Vertical,SubVertical,City Location,Investors Name,InvestmentnType,Amount in USD,Remarks
0,1,09/01/2020,BYJU’S,E-Tech,E-learning,Bengaluru,Tiger Global Management,Private Equity Round,"20,00,00,000",NaN
1,2,13/01/2020,Shuttl,Transportation,App based shuttle service,Gurgaon,Susquehanna Growth Equity,Series C,"80,48,394",NaN
2,3,09/01/2020,Mamaearth,E-commerce,Retailer of baby and toddler products,Bengaluru,Sequoia Capital India,Series B,"1,83,58,860",NaN
3,4,02/01/2020,https://www.wealthbucket.in/,FinTech,Online Investment,New Delhi,Vinod Khatumal,Pre-series A,"30,00,000",NaN
4,5,02/01/2020,Fashor,Fashion and Apparel,Embroiled Clothes For Women,Mumbai,Sprout Venture Partners,Seed Round,"18,00,000",NaN
...,...,...,...,...,...,...,...,...,...,...
3039,3040,29/01/2015,Printvenue,NaN,NaN,NaN,Asia Pacific Internet Group,Private Equity,"45,00,000",NaN
3040,3041,29/01/2015,Graphene,NaN,NaN,NaN,KARSEMVEN Fund,Private Equity,"8,25,000",Govt backed VC Fund
3041,3042,30/01/2015,Mad Street Den,NaN,NaN,NaN,"Exfinity Fund, GrowX Ventures.",Private Equity,"15,00,000",NaN
3042,3043,30/01/2015,Simplotel,NaN,NaN,NaN,MakeMyTrip,Private Equity,NaN,"Strategic Funding, Minority stake"


In [8]:
{column: len(df[column].unique()) for column in df.columns}

{'Sr No': 3044,
 'Date dd/mm/yyyy': 1035,
 'Startup Name': 2459,
 'Industry Vertical': 822,
 'SubVertical': 1943,
 'City  Location': 113,
 'Investors Name': 2413,
 'InvestmentnType': 56,
 'Amount in USD': 472,
 'Remarks': 73}

In [9]:
# Drop id and high cardinality columns
df = df.drop(['Sr No', 'Startup Name', 'SubVertical', 'Investors Name'], axis=1)

In [10]:
df

,Date dd/mm/yyyy,Industry Vertical,City Location,InvestmentnType,Amount in USD,Remarks
0,09/01/2020,E-Tech,Bengaluru,Private Equity Round,"20,00,00,000",NaN
1,13/01/2020,Transportation,Gurgaon,Series C,"80,48,394",NaN
2,09/01/2020,E-commerce,Bengaluru,Series B,"1,83,58,860",NaN
3,02/01/2020,FinTech,New Delhi,Pre-series A,"30,00,000",NaN
4,02/01/2020,Fashion and Apparel,Mumbai,Seed Round,"18,00,000",NaN
...,...,...,...,...,...,...
3039,29/01/2015,NaN,NaN,Private Equity,"45,00,000",NaN
3040,29/01/2015,NaN,NaN,Private Equity,"8,25,000",Govt backed VC Fund
3041,30/01/2015,NaN,NaN,Private Equity,"15,00,000",NaN
3042,30/01/2015,NaN,NaN,Private Equity,NaN,"Strategic Funding, Minority stake"


In [11]:
df.isna().mean()

Date dd/mm/yyyy      0.000000
Industry Vertical    0.056176
City  Location       0.059133
InvestmentnType      0.001314
Amount in USD        0.315375
Remarks              0.862352
dtype: float64

In [12]:
# Clearn \\\\xc2\\\\xa0 examples
df = df.applymap(lambda x: x.replace('\\\\xc2\\\\xa0', '') if type(x) == str else x)

C:\Users\dukea_6wbfoy1\AppData\Local\Temp\ipykernel_5248\1057971691.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.replace('\\\\xc2\\\\xa0', '') if type(x) == str else x)


In [13]:
# Drop missing target rows
missing_target_rows = df[df['Amount in USD'].isna()].index
df = df.drop(missing_target_rows, axis=0).reset_index(drop=True)

In [14]:
df.isna().mean()

Date dd/mm/yyyy      0.000000
Industry Vertical    0.062860
City  Location       0.064779
InvestmentnType      0.001440
Amount in USD        0.000000
Remarks              0.836372
dtype: float64

In [15]:
# Drop columns with more than 25% missing values
df = df.drop('Remarks', axis=1)

In [16]:
df

,Date dd/mm/yyyy,Industry Vertical,City Location,InvestmentnType,Amount in USD
0,09/01/2020,E-Tech,Bengaluru,Private Equity Round,"20,00,00,000"
1,13/01/2020,Transportation,Gurgaon,Series C,"80,48,394"
2,09/01/2020,E-commerce,Bengaluru,Series B,"1,83,58,860"
3,02/01/2020,FinTech,New Delhi,Pre-series A,"30,00,000"
4,02/01/2020,Fashion and Apparel,Mumbai,Seed Round,"18,00,000"
...,...,...,...,...,...
2079,28/01/2015,NaN,NaN,Private Equity,"25,00,000"
2080,29/01/2015,NaN,NaN,Private Equity,"45,00,000"
2081,29/01/2015,NaN,NaN,Private Equity,"8,25,000"
2082,30/01/2015,NaN,NaN,Private Equity,"15,00,000"


In [17]:
# Rename the columns
df.columns = ['Date', 'Industry Vertical', 'City Location', 'InvestmentType', 'Amount in USD']

In [18]:
# Fill categorical missing values with most frequent occurrence
for column in ['Industry Vertical', 'City Location', 'InvestmentType']:
    df[column] = df[column].fillna(df[column].mode()[0])

In [19]:
df.isna().sum()

Date                 0
Industry Vertical    0
City Location        0
InvestmentType       0
Amount in USD        0
dtype: int64

In [20]:
pd.to_datetime?

Signature:
pd.to_datetime(
    arg: 'DatetimeScalarOrArrayConvertible | DictConvertible',
    errors: 'DateTimeErrorChoices' = 'raise',
    dayfirst: 'bool' = False,
    yearfirst: 'bool' = False,
    utc: 'bool' = False,
    format: 'str | None' = None,
    exact: 'bool | lib.NoDefault' = <no_default>,
    unit: 'str | None' = None,
    infer_datetime_format: 'lib.NoDefault | bool' = <no_default>,
    origin: 'str' = 'unix',
    cache: 'bool' = True,
) -> 'DatetimeIndex | Series | DatetimeScalar | NaTType | None'
Docstring:
Convert argument to datetime.

This function converts a scalar, array-like, :class:`Series` or
:class:`DataFrame`/dict-like to a pandas datetime object.

Parameters
----------
arg : int, float, str, datetime, list, tuple, 1-d array, Series, DataFrame/dict-like
    The object to convert to a datetime. If a :class:`DataFrame` is provided, the
    method expects minimally the following columns: :const:`"year"`,
    :const:`"month"`, :const:`"day"`. The column "year"
 

In [21]:
# Working on Date column
df['Date'] = pd.to_datetime(df['Date'], dayfirst=True, errors='coerce')

In [21]:
# # Clean Date column
# df['Date'] = df['Date'].replace({
#     '05/072018': '05/07/2018',
#     "01/07/015": "01/07/2015",
#     "12/05.2015": "12/05/2015",
#     "13/04.2015": "13/04/2015",
#     "15/01.2015": "15/01/2015",
#     "22/01//2015": "22/01/2015"
# })

In [23]:
# Extract Date features
df['Year'] = df['Date'].apply(lambda x: x.year)
df['Month'] = df['Date'].apply(lambda x: x.month)
df['Day'] = df['Date'].apply(lambda x: x.day)
df = df.drop('Date', axis=1)

In [25]:
df

,Industry Vertical,City Location,InvestmentType,Amount in USD,Year,Month,Day
0,E-Tech,Bengaluru,Private Equity Round,"20,00,00,000",2020.0,1.0,9.0
1,Transportation,Gurgaon,Series C,"80,48,394",2020.0,1.0,13.0
2,E-commerce,Bengaluru,Series B,"1,83,58,860",2020.0,1.0,9.0
3,FinTech,New Delhi,Pre-series A,"30,00,000",2020.0,1.0,2.0
4,Fashion and Apparel,Mumbai,Seed Round,"18,00,000",2020.0,1.0,2.0
...,...,...,...,...,...,...,...
2079,Consumer Internet,Bangalore,Private Equity,"25,00,000",2015.0,1.0,28.0
2080,Consumer Internet,Bangalore,Private Equity,"45,00,000",2015.0,1.0,29.0
2081,Consumer Internet,Bangalore,Private Equity,"8,25,000",2015.0,1.0,29.0
2082,Consumer Internet,Bangalore,Private Equity,"15,00,000",2015.0,1.0,30.0


In [26]:
# Clean Target column
df['Amount in USD'] = df['Amount in USD'].apply(lambda x: x.replace(',', ''))

In [27]:
z = df['Amount in USD'].copy()
for i, x in z.items():
    try:
        x = float(x)
    except:
        print(x)

undisclosed
unknown
Undisclosed
undisclosed
undisclosed
14342000+
Undisclosed
Undisclosed
N/A
N/A
N/A
N/A


In [28]:
df['Amount in USD'] = df['Amount in USD'].replace({
    'undisclosed': np.nan,
    'unknown': np.nan,
    'Undisclosed': np.nan,
    'N/A': np.nan,
    '14342000+': '14342000'
})

In [29]:
for i, x in df['Amount in USD'].items():
    try:
        x = float(x)
    except:
        print(x)

In [30]:
df.isna().sum()

Industry Vertical     0
City Location         0
InvestmentType        0
Amount in USD        11
Year                  6
Month                 6
Day                   6
dtype: int64

In [31]:
# Drop rows with missing target values
df = df.drop(df[df['Amount in USD'].isna()].index, axis=0)

In [32]:
df.isna().sum()

Industry Vertical    0
City Location        0
InvestmentType       0
Amount in USD        0
Year                 6
Month                6
Day                  6
dtype: int64

In [33]:
# Fill missing values with mean
for column in ['Year', 'Month', 'Day']:
    df[column] = df[column].fillna(df[column].mean())

In [34]:
df.isna().sum()

Industry Vertical    0
City Location        0
InvestmentType       0
Amount in USD        0
Year                 0
Month                0
Day                  0
dtype: int64

In [35]:
# Split df into X and y
y = df['Amount in USD'].copy()
X = df.drop('Amount in USD', axis=1)

In [36]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, shuffle=True, random_state=1)

In [37]:
X_train

,Industry Vertical,City Location,InvestmentType,Year,Month,Day
931,eCommerce,Noida,Private Equity,2016.0,10.0,4.0
1115,Consumer Internet,Bangalore,Seed Funding,2016.0,7.0,21.0
1066,Consumer Internet,Mumbai,Private Equity,2016.0,8.0,29.0
167,Consumer Internet,Bengaluru,Seed/ Angel Funding,2018.0,8.0,8.0
1703,on-demand delivery service,Gurgaon,Seed Funding,2015.0,8.0,17.0
...,...,...,...,...,...,...
967,eCommerce,Ahmedabad,Private Equity,2016.0,10.0,26.0
912,Consumer Internet,Mumbai,Private Equity,2016.0,11.0,24.0
1103,eCommerce,New Delhi,Seed Funding,2016.0,7.0,15.0
242,Finance,Chennai,Seed / Angel Funding,2018.0,5.0,2.0


In [38]:
y_train

931     4200000
1115     595000
1066    3000000
167     4000000
1703     310000
         ...   
967     1000000
912     4000000
1103     250000
242      450000
1068    1000000
Name: Amount in USD, Length: 1451, dtype: object

### Building the Pipeline

In [42]:
# tf.keras.wrappers.scikit_learn.KerasRegressor  / deprecated

In [43]:
!pip install scikeras -q

In [44]:
from scikeras.wrappers import KerasRegressor

In [49]:
def build_model():
    inputs = tf.keras.Input(shape=(532,))
    x = tf.keras.layers.Dense(128, activation='relu')(inputs)
    x = tf.keras.layers.Dense(128, activation='relu')(x)
    outputs = tf.keras.layers.Dense(1, activation='linear')(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    model.compile(
        optimizer='adam',
        loss='mse'
    )

    return model

In [58]:
nominal_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocessor = ColumnTransformer(transformers=[
    ('nominal', nominal_transformer, ['Industry Vertical', 'City Location', 'InvestmentType'])
], remainder='passthrough')

regressor = KerasRegressor(build_model)

model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('scaler', StandardScaler()),
    ('regressor', regressor)
])

In [59]:
# model.fit_transform(X_train).shape

### Training

In [60]:
model.fit(X_train, 
          y_train,
         regressor__validation_split=0.2,
         regressor__batch_size=32,
         regressor__epochs=100,
         regressor__callbacks=[
             tf.keras.callbacks.EarlyStopping(
                 monitor='val_loss',
                 patience=3,
                 restore_best_weights=True
             )
         ])

Epoch 1/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 17648135509639168.0000 - val_loss: 29475037702520832.0000
Epoch 2/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 17648135509639168.0000 - val_loss: 29475035555037184.0000
Epoch 3/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 17648135509639168.0000 - val_loss: 29475029112586240.0000
Epoch 4/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 17648120477253632.0000 - val_loss: 29475014080200704.0000
Epoch 5/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 17648103297384448.0000 - val_loss: 29474986162913280.0000
Epoch 6/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 17648070011387904.0000 - val_loss: 29474936770789376.0000
Epoch 7/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 17648022766747648.0000 - val_loss: 29474872346279936.0000
Epoch 8/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 17647948678561792.0000 - val_loss: 29474780004483072.0000
Epoch 9/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1764

,steps,"[('preprocessor', ...), ('scaler', ...), ...]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('nominal', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


### Results

In [65]:
y_test = y_test.astype(float)
y_test

731      1000000.0
355      1930000.0
107     26000000.0
1087      297000.0
1774    30000000.0
           ...    
1971     1300000.0
1604    40000000.0
2056      290000.0
1402     1800000.0
1863     1000000.0
Name: Amount in USD, Length: 622, dtype: float64

In [66]:
y_pred = model.predict(X_test)

rmse = np.sqrt(np.mean((y_test - y_pred)**2))

20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 


In [68]:
print("Test RMSE: {:.2f}".format(rmse))

Test RMSE: 56290204.44


In [69]:
np.mean(y_test)

np.float64(14110447.28778135)

In [70]:
r2 = r2_score(y_test, y_pred)

print("Test R^2 Score: {:.5f}".format(r2))

Test R^2 Score: -0.01817
